In [1]:
from __init__ import *

In [1]:
from trpca import trpca
from t_prod import t_prod

In [2]:
import numpy as np

In [2]:
#generate problem: recover L_0 (and C_0) when given M_0

n = 25    #size of each dimension (3 dimension tensor)
rank = 3  #rank of the tensor
rho = .1  #proportion of corrupted entries

A = np.random.normal(0,1/n,(n,rank,n))
B = np.random.normal(0,1/n,(rank,n,n))
L_0 = t_prod(A,B)

S_0 = np.random.binomial(1,rho, L_0.shape)
N   = np.random.normal(0,1/n, L_0.shape)
C_0 = S_0 * N

M_0 = L_0 + C_0

In [3]:
get_frontal_faces(np.ones((4,4,2,2)))

{(0, 0), (0, 1), (1, 0), (1, 1)}

In [4]:
#set reguralisation
dims = M_0.shape
n_min = min(dims[0], dims[1])
reg = np.sqrt( n_min / np.prod(dims))

In [5]:
#apply algorithm
L , C = trpca(M_0, reg)

In [6]:
#test effectiveness
L_error = np.linalg.norm(L - L_0)/np.linalg.norm(L_0)
C_error = np.linalg.norm(C - C_0)/np.linalg.norm(C_0)
print(L_error, C_error)

1.5113407224542788e-08 1.093985741825828e-08
